In [98]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from string import punctuation
import re
from nltk.stem import SnowballStemmer
from autocorrect import Speller
from collections import Counter 

In [58]:
contractions_dict = {     
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I had",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "iit will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that had",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there had",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they had",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [118]:
expand_contractions(ds.lyrics[0],contractions_dict)

'[Missy]\nThis is for my ghetto motherfuckers, uh\n\n[Nas]\nUh, forty side felony\nFelony, misdemeanor\nThe charge is murder\nEscobar CBR bikes\nI am switching gears\nHeadlights, shine so bright\nBitches freeze like deers\nThem fiends want that deep boy\nFeds send in a decoy, pack that heat boy\nPush ya where ya rest in peace boy\nGet your mamas house shot up\nBodies all chopped up\nWhen them bodies pop up, I am not getting locked up\nMy Bentley cruise the block, with the sun roof top\nHood rats jumpin on my jock cause I blew up the spot\nCrushin your Benz, crushin your Navigator system\nMy QB piece make you all niggas tuck your shit in\nit is Nas in your area, Queens bout to tear it up\nBraveheart you all scared of us, real niggas, they be us\n\n[Missy & Lil Mo]\nwhat is your name?, cause I am impressed\nCan you treat me good, I will not settle for less\nYou a hot boy, a rock boy\nA fun toy, tote a Glock boy\nWhere you live, is it by yourself?\nCan I move with you, do you need some he

In [102]:
stopword = stopwords.words('english')
wordnet_lemmatizer = WordNetLemmatizer()
snowball_stemmer = SnowballStemmer('english')
punct=punctuation+'’'+'“'+'”'

def expand_contractions(text, contractions_dict):
    contractions_pattern = re.compile('({})'.format('|'.join(contractions_dict.keys())),
                                      flags=re.IGNORECASE | re.DOTALL)

    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contractions_dict.get(match) \
            if contractions_dict.get(match) \
            else contractions_dict.get(match.lower())
        expanded_contraction = expanded_contraction
        return expanded_contraction

    expanded_text = contractions_pattern.sub(expand_match, text)
    expanded_text = re.sub("'", "", expanded_text)
    return expanded_text

def autospell(text):
    """
    correct the spelling of the word.
    """
    spell = Speller(lang='en',fast=True)
    spells = [spell(w) for w in (nltk.word_tokenize(text))]
    return " ".join(spells)



def nlp_clean(text):
    text = expand_contractions(text,contractions_dict)
    text = autospell(text)
    text = ' '.join([w.lower().strip('intro').strip('chorus') for w in nltk.word_tokenize(text)])
    text = re.sub('<[^<]+?>','', text)
    text = ''.join(c for c in text if not c.isdigit())
    text = ''.join(c for c in text if c not in punct)
    word_tokens = nltk.word_tokenize(text)
    word_tokens = [wordnet_lemmatizer.lemmatize(word) for word in word_tokens]
    word_tokens = [snowball_stemmer.stem(word) for word in word_tokens]
    result = [word for word in word_tokens if word not in stopword]
    
    return result



In [112]:

data=df.Lyrics.apply(nlp_clean)


TypeError: expected string or bytes-like object

In [120]:
ds=ds.dropna()

data2=df.Lyrics.apply(nlp_clean)

,Unnamed: 0,title,artist,reached_number_one,lyrics
0,0,Hot Boyz,"Missy Elliott featuring Nas, Eve and Q-Tip","November 27, 1999","[Missy]\nThis is for my ghetto motherfuckers, ..."
1,1,Whistle While You Twurk,Ying Yang Twins,"April 1, 2000",Whistle while you twurk (whistle)\nGo head and...
3,3,Country Grammar (Hot Shit),Nelly,"May 27, 2000",[Intro]\nHot Shit (Hot Shit)\n\n[Chorus]\nMmmm...
4,4,Flamboyant,Big L,"June 24, 2000","[Intro]\nYeah, yeah! Big L, Corleone\nMy nigga..."
5,5,Callin' Me,Lil Zane featuring 112,"July 15, 2000",[Intro: Lil Zane & (Slim)]\nI live the life of...
...,...,...,...,...,...
88,88,Boom Boom Pow,The Black Eyed Peas,"June 6, 2009",2000\n#1 Breathe by Faith Hill vs. #4 I Wanna ...
89,89,Best I Ever Had,Drake,"June 13, 2009","[Intro]\nYou know, a lot of girls be\nThinkin'..."
90,90,Run This Town,Jay-Z featuring Rihanna and Kanye West,"September 26, 2009",[Produced by Kanye West & No I.D]\n\n[Intro: R...
91,91,Forever,"Drake featuring Kanye West, Lil Wayne and Eminem","November 14, 2009",[Chorus: Drake]\nIt may not mean nothin' to y'...


In [104]:
data

0      [malcolm, x, agre, ght—all, f, e, peak, ave, a...
1      [de, la, l, yea, yea, yea, yea, list, p, eck, ...
2      [vers, e, ype, f, guy, ntrol, girl, buy, e, lo...
3      [produc, e, bomb, quad, ye, educ, equip, prepa...
4      [eadi, f, astafara, nni, mazda, fa, drop, e, l...
                             ...                        
183    [jayz, afel, buff, de, wizz, bea, alm, jigga, ...
184    [wha, p, eg, egrett, boy, warr, g, know, wha, ...
185    [puff, wa, mebodi, w, g, love, f, mebodi, love...
186    [money, wha, p, le, wha, p, babi, money, di, g...
187    [mi, f, ghe, motherfuck, forti, ide, feloni, f...
Name: Lyrics, Length: 187, dtype: object

In [101]:
word_count=data.apply(Counter).sum()
for w in sorted(word_count, key=word_count.get, reverse=True):
        print (w, word_count[w])


wa 1527
like 1459
get 1318
got 1225
“ 1020
” 1015
know 968
one 890
na 866
go 801
come 765
de 718
make 708
time 696
say 688
man 667
see 616
becaus 548
featur 538
boy 531
love 502
live 501
would 501
ya 491
vers 485
u 480
want 470
back 439
yeah 434
said 434
rap 415
let 409
origin 409
version 404
shout 401
take 398
rock 398
feel 394
yo 381
fuck 379
day 376
girl 366
play 357
em 356
never 355
think 352
right 350
look 348
mme 346
jam 345
gon 345
could 343
big 342
keep 340
peopl 331
def 330
call 329
caus 329
beat 325
tri 323
wan 320
good 316
shit 313
life 311
rhyme 310
verdurin 310
way 306
babi 302
ta 300
even 300
tell 298
word 292
littl 290
ha 289
chorus 285
new 285
onli 284
money 279
give 276
name 270
need 259
thing 254
check 252
wild 246
nigga 245
‘ 236
real 234
nigra 230
brother 227
whi 227
hook 227
put 227
x 223
two 223
fresh 215
world 214
funki 213
g 213
made 212
friend 209
mind 208
might 208
oh 206
still 206
daddi 205
long 204
cambrem 204
first 201
hit 199
without 197
bad 196
c 192
eye 

refer 28
tongu 28
paint 28
age 28
reflect 28
starr 28
realis 28
sudden 28
habit 28
baron 28
chain 27
justic 27
apart 27
record 27
alma 27
met 27
ol 27
carri 27
finger 27
um 27
demo 27
earth 27
nasti 27
enter 27
month 27
hors 27
scout 27
bare 27
famous 27
ray 27
draw 27
sourc 27
intern 27
rocafel 27
beach 27
pretend 27
bernard 27
glanc 27
liftboy 27
impress 26
someon 26
deni 26
control 26
receiv 26
glad 26
dumb 26
loos 26
near 26
surpris 26
flex 26
zone 26
scare 26
slick 26
sens 26
date 26
tabl 26
outsid 26
compani 26
shootout 26
joy 26
pimp 26
sir 26
link 26
card 26
expect 26
rick 26
giddi 26
qtip 26
starski 26
elster 26
annoy 26
spoke 26
lock 25
memori 25
switch 25
battl 25
sincer 25
horn 25
larg 25
easi 25
eight 25
techniqu 25
wax 25
solo 25
ground 25
suggest 25
sermon 25
hardcor 25
z 25
either 25
moon 25
sole 25
wet 25
exist 25
bastard 25
truss 25
chrysali 25
theme 25
junior 25
zaki 25
stetsason 25
saltnepa 25
california 25
general 25
exclaim 25
instanc 25
blame 24
wall 24
cloth 24


plead 8
eas 8
motto 8
captiv 8
dynamit 8
sequel 8
river 8
execut 8
lava 8
gem 8
voodoo 8
tactic 8
fortifi 8
juvenil 8
thievz 8
scorn 8
mysteri 8
wo 8
perman 8
maladi 8
physic 8
appar 8
incarvill 8
resign 8
gradual 8
neighbour 8
neighbourhood 8
newcom 8
profession 8
seiz 8
usual 8
profess 8
apolog 8
trembl 8
angri 8
scent 8
consult 8
dash 8
indulg 8
particular 8
major 8
interpret 8
emerg 8
dowag 8
neurasthenia 8
convey 8
defend 8
examin 8
highest 8
cast 8
indiffer 8
silent 8
signal 8
ventur 8
writer 8
extraordinari 8
refin 8
farther 8
remot 8
post 8
sensit 8
assign 8
putbus 8
tomato 8
fond 8
chantepi 8
jamal 8
ibwin 8
snoopi 8
komradz 8
figgaz 8
expens 8
cooool 8
bum 7
uniti 7
pig 7
difficulti 7
heed 7
selfdestruct 7
evolv 7
english 7
wink 7
bandit 7
equip 7
careless 7
reli 7
deliv 7
mellow 7
tempo 7
dissin 7
nightmar 7
rid 7
disgust 7
gras 7
mania 7
hotti 7
commit 7
sale 7
saddl 7
flight 7
bob 7
rope 7
tom 7
harri 7
gogo 7
im 7
limp 7
higher 7
proceed 7
betcha 7
fetch 7
drain 7
bail 7


resum 4
villeparisi 4
horizon 4
slope 4
nicknam 4
advis 4
mute 4
restor 4
wholli 4
abandon 4
briefli 4
brothel 4
insol 4
coars 4
née 4
print 4
joyous 4
format 4
diningroom 4
reward 4
behav 4
attain 4
momentari 4
saintfrichoux 4
summon 4
liber 4
academi 4
gamma 4
behalf 4
assert 4
bicycl 4
maid 4
withdrew 4
selfesteem 4
pronoun 4
temper 4
twitter 4
wellbr 4
absenc 4
particl 4
ginest 4
unlik 4
disturb 4
condit 4
treatment 4
fasten 4
literatur 4
wretch 4
beforehand 4
falsehood 4
spoil 4
affirm 4
accus 4
actress 4
detach 4
depriv 4
healthi 4
sooner 4
habitu 4
injuri 4
hypothesi 4
harshest 4
sweep 4
endow 4
incident 4
sisterinlaw 4
regiment 4
compromis 4
sharper 4
glaze 4
gull 4
superb 4
aesthet 4
pinch 4
princip 4
drawback 4
recaptur 4
mediocr 4
calcul 4
precious 4
montesquieu 4
chiefli 4
appropri 4
forward 4
borrow 4
plebeian 4
enlarg 4
confirm 4
ident 4
mole 4
varieti 4
mould 4
furtiv 4
feminin 4
apt 4
bishop 4
friendli 4
uneasi 4
inasmuch 4
generos 4
consequ 4
corridor 4
percept 4
modes

depart 2
crisp 2
interven 2
pebbl 2
traitor 2
orgi 2
neat 2
cb 2
rico 2
tab 2
poll 2
corpor 2
pusher 2
user 2
strategi 2
idol 2
dwayn 2
sunni 2
jama 2
rie 2
june 2
comb 2
rum 2
jockin 2
pessimist 2
forefath 2
wisdom 2
aerl 2
stiggedi 2
drippedydrop 2
spiggedyspark 2
willow 2
sheriff 2
nestl 2
cocoa 2
nincompoop 2
decker 2
babaloo 2
hockey 2
cone 2
per 2
sterl 2
sober 2
kicker 2
senil 2
verbatim 2
crab 2
pool 2
daytim 2
uniform 2
scope 2
dadoubledymac 2
script 2
research 2
scientist 2
technic 2
obsolet 2
showbiz 2
cornwal 2
honda 2
oz 2
misfit 2
outfit 2
anatomi 2
miggedymak 2
banana 2
rocki 2
giggedygot 2
tuna 2
trot 2
bunker 2
foster 2
dunk 2
hut 2
caviti 2
market 2
teepe 2
insert 2
paragraph 2
serial 2
diet 2
circumfer 2
knuckl 2
incom 2
everclear 2
muscl 2
lightn 2
blacula 2
guilt 2
thorn 2
peach 2
ninetytwo 2
delawar 2
seduct 2
dice 2
trickin 2
suburban 2
foreplay 2
wha 2
outa 2
inject 2
picket 2
tube 2
champa 2
bunni 2
fullest 2
inmat 2
raggamuffin 2
aspirin 2
fastest 2
outti 2
la

mela 1
ballerina 1
amanda 1
yanke 1
tue 1
evermor 1
hep 1
circular 1
birkin 1
watskyrelief 1
demeanor 1
battleship 1
hairdress 1
confessor 1
tester 1
boobi 1
pic 1
unclean 1
grassi 1
kennedi 1
kc 1
uranus 1
lennon 1
audrey 1
hepburn 1
campaign 1
belafont 1
hetrix 1
youtub 1
shoyu 1
voter 1
polli 1
bonnet 1
stopdropandrol 1
amboli 1
empow 1
jackal 1
jade 1
jaundic 1
sandman 1
whackest 1
jolli 1
weezer 1
judicar 1
junkey 1
jubon 1
poop 1
kamala 1
jumanji 1
kappa 1
parappa 1
genom 1
keypad 1
spitter 1
layerag 1
fridg 1
leghold 1
lentisk 1
scarecrow 1
earlob 1
paso 1
encinita 1
lagunita 1
alameda 1
emeryvill 1
iliad 1
lilypad 1
foggi 1
unsur 1
darn 1
healer 1
shield 1
tooshi 1
copho 1
hillel 1
datum 1
gramcb 1
cal 1
matia 1
havana 1
owl 1
barn 1
emerson 1
glenn 1
syndrom 1
chalic 1
mullen 1
marissa 1
playlist 1
ninjasmooth 1
gong 1
sebastian 1
scorpion 1
themat 1
thesbian 1
vegan 1
volt 1
weakest 1
rapunzel 1
haliburton 1
someday 1
ascens 1
upto 1
stag 1
dud 1
crude 1
clientel 1
stofer 1
m

holland 1
vermeer 1
meaningless 1
superwagn 1
satieti 1
bayreuth 1
massenet 1
microb 1
corpuscl 1
ensur 1
untutor 1
precursor 1
fratern 1
turner 1
flaubert 1
predilect 1
auspex 1
commend 1
cushion 1
quaint 1
paradox 1
deliver 1
shone 1
inhal 1
dilat 1
fide 1
lifegiv 1
hirsuta 1
trumpetton 1
delirium 1
supersecret 1
armour 1
demosthen 1
tidemark 1
foam 1
steep 1
outgrown 1
vowel 1
nonceaux 1
chenouvilleand 1
jargon 1
naturalis 1
rolîan 1
implement 1
transmut 1
caro 1
brunetier 1
lamoureux 1
volatil 1
parentsinlaw 1
forefing 1
halfshut 1
extol 1
alleg 1
doncièr 1
carnbrem 1
evas 1
gnaw 1
sistersinlaw 1
satiat 1
ranunculus 1
protrud 1
xenophobia 1
beaten 1
retreat 1
x——may 1
penchant 1
irrefut 1
testimoni 1
crisenoy 1
premis 1
kindest 1
coastlin 1
silveri 1
angélus 1
ratherpelléa 1
faintest 1
raucous 1
deglutit 1
ostent 1
unsuccess 1
grimac 1
seventeenth 1
masteri 1
haze 1
duski 1
gnash 1
harttissttick 1
crozier 1
overload 1
adsum 1
subtleti 1
trumperi 1
solemn 1
resort 1
atrophi 1
cursor

memento 1
bequeath 1
agar 1
watercolour 1
enrich 1
benefici 1
arisen 1
astut 1
enrag 1
draught 1
liningcloth 1
pane 1
counciloffic 1
resolut 1
compassion 1
worn 1
threadbar 1
quilt 1
impressionist 1
tradespeopl 1
rheim 1
organloft 1
pulpit 1
misericord 1
massacr 1
recours 1
paralyt 1
stricken 1
agraphia 1
zealous 1
margin 1
laudatori 1
defianc 1
diminuendo 1
outran 1
fertil 1
impel 1
noun 1
amitié 1
timehonour 1
discreet 1
herald 1
rewrit 1
kiel 1
usurp 1
annex 1
alsacelorrain 1
tschudi 1
elstir 1
monarch 1
naval 1
roi 1
soleil 1
republ 1
driblet 1
eulenburg 1
culprit 1
scaffold 1
germani 1
mediatis 1
durchlaucht 1
forbad 1
withheld 1
thérèse 1
espinay 1
commerc 1
forebear 1
émigré 1
epigram 1
unambiti 1
imparti 1
hypocrisi 1
arpajon 1
sonnet 1
sunk 1
ingenu 1
fluenci 1
acoust 1
miser 1
sophistri 1
tacit 1
emigr 1
disparag 1
infirm 1
aumal 1
speechless 1
dissert 1
burial 1
greatgreatgrandmoth 1
hassock 1
bourgogn 1
battlecri 1
passav 1
ead 1
poland 1
hungari 1
palatin 1
savoycarignano 

dolc 1
blimp 1
goldi 1
cuddi 1
hooti 1
ddi 1
yacht 1
thisll 1
doa 1
poppa 1
ea 1
flagrant 1
holster 1
pizat 1
thizat 1
trizack 1
flizat 1
lindbergh 1
jacuzzi 1
maganoo 1
fiveoh 1
offtim 1
marriott 1
doubledeck 1
itwit 1
wack 1
stickup 1
wallet 1
testicl 1
midoubl 1
sye 1
보지 1
못한게 1
많지 1
신어 1
야될 1
나이키도 1
많아 1
구름 1
위를 1
걷는 1
꿈꿨지 1
온 1
세상에는 1
내노래가 1
모두 1
틀어지고 1
나를 1
싫어하는 1
사람들도 1
결국 1
인정하고 1
목소리에 1
기대면서 1
정말 1
가고 1
있는데 1
안개가 1
낄땐 1
한계를 1
묻네 1
남들보다 1
조금 1
더더 1
속도를 1
낼땐 1
내게 1
쏴주는 1
눈초리는 1
완전 1
총알같에 1
누가 1
날 1
과거는 1
지워 1
하고 1
싶은대로 1
신경 1
안써 1
무채색 1
사이로 1
보이네 1
바로 1
널 1
사로잡네 1
너네완 1
다른 1
해답은 1
궁금하면 1
직접 1
찾아가 1
부리는 1
욕심에 1
멍청하게 1
말해줘도 1
그게 1
틀렸다는 1
것은 1
아냐 1
자기 1
보고 1
밤에 1
그리는 1
꿈속을 1
헤엄치는 1
이무기가 1
용이 1
되며 1
뭐든지 1
첨부터 1
있을순 1
지구 1
제일 1
금수저라고 1
꿈을 1
그리고 1
아래서부터 1
흘린 1
자만 1
아는 1
뼈에 1
새긴 1
배움 1
수많은 1
사람들이 1
포기해 1
사람들은 1
쉽게 1
변하지 1
일이 1
싫은 1
너는 1
일을 1
옮기네 1
변하는건 1
어두운 1
길을 1
밝히네 1
거기에 1
항상 1
배고파 1
신선한 1
메뉴를 1
쭉 1
골라 1
면보다는 1
새우초밥 1
사이다 1
보다는 1
콜라 1
소프트보단 1
메로나 1
하지만 1
여자는 1
몰라 1
신나는 1
분위기에 1
주체를

In [79]:
df

,Unnamed: 0,Song Title,Artist,Date Reached #1,Lyrics
0,0,Self-Destruction,Stop the Violence Movement,"March 11, 1989",[Intro: Malcolm X]\nWe all agree tonight—all o...
1,1,Me Myself and I,De La Soul,"May 20, 1989","[Intro: De La Soul]\nYeah, yeah, yeah, yeah\nC..."
2,2,I'm That Type of Guy,LL Cool J,"July 15, 1989",[Verse 1]\nYou're the type of guy that can't c...
3,3,Fight the Power,Public Enemy,"July 29, 1989",[Produced by The Bomb Squad]\n\n[Intro]\nYet o...
4,4,It's Funky Enough,The D.O.C.,"September 9, 1989",[Intro]\nY'all ready for this\nRastafara Sonna...
...,...,...,...,...,...
183,183,Jigga My Nigga,Jay-Z,"September 11, 1999","[Intro: Jay-Z]\nRoc-A-Fella, Ruff Ryders, Swiz..."
184,184,I Want It All,Warren G featuring Mack 10,"September 25, 1999",[Intro]\nWhat's up negros and negrettes?\nIt's...
185,185,Satisfy You,Puff Daddy featuring R. Kelly,"October 23, 1999",[Puffy]\nAll I want is somebody who's gonna lo...
186,186,"4, 5, 6",Solé featuring JT Money and Kandi,"November 20, 1999","[JT Money]\nYo, what's up?\n[Sole']\nWhat's up..."
